Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 01 - Natural Language Processing (NLP)

Im Folgenden wird ein [CoNLL-2003](https://huggingface.co/datasets/imdb) Datensatz verwendet.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [ ]:
dataset

In [ ]:
dataset["train"][0]

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

## Tokenizer und Model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
def tokenize_and_align_labels(ds):
    tokenized_inputs = tokenizer(ds["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(ds["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx: 
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
id2label = {i: name for i, name in enumerate(label_list)}
label2id = {name: i for i, name in enumerate(label_list)}

### Modeling

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(label_list), id2label=id2label, label2id=label2id)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_dataset["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

In [ ]:
from tensorflow.keras.optimizers import Adam

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

model.compile(optimizer=Adam(learning_rate=5e-5))
model.fit(tf_train_set, validation_data=tf_validation_set, epochs=1, callbacks=[metric_callback])

# Evaluate

In [ ]:
inputs = tokenizer(dataset["test"][0]["tokens"], return_tensors="tf", truncation=True, is_split_into_words=True)
logits = model(**inputs).logits
inputs

In [ ]:
from tensorflow.math import argmax

predicted_token_class_ids = argmax(logits, axis=-1)


predicted_token_class = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]
predicted_token_class

In [ ]:
dataset["test"][0]["tokens"]

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
sep = [i for i, t in enumerate(tokens) if "##" in t]

" ".join(tokens).replace(" ##", "")

In [ ]:
token_class = ""
for i, p in enumerate(predicted_token_class):
    if i + 1 in sep:
        token_class += p + ","
        continue
    token_class += p + " "

token_class

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 